**NOTE** Use teneva version 0.9.14 !!!

---

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

# Code

In [2]:
def _ones(n):
    return np.ones((n, 1), dtype=int)

In [3]:
def _range(n):
    return np.arange(n).reshape(-1, 1)

In [4]:
def optima_tt_beam_left(Y, k=100, ort_num=-1, ret_all=False):
    ort_num = ort_num if ort_num >= 0 else len(Y)-1
    Z = teneva.orthogonalize(Y, ort_num)
    
    G = Z[-1]
    r1, n, r2 = G.shape
    Q = G.reshape(r1, n)
    I = _range(n)

    for G in Z[:-1][::-1]:
        r1, n, r2 = G.shape
        Q = np.einsum('qir,rj->qij', G, Q).reshape(r1, -1)
        
        I1 = np.kron(_range(n), _ones(I.shape[0]))
        I2 = np.kron(_ones(n), I)
        I = np.hstack((I1, I2))
        
        norms = np.sum(Q**2, axis=0)
        ind = np.argsort(norms)[:-(k+1):-1]
        I = I[ind, :]
        Q = Q[:, ind]

    return I if ret_all else I[0]

In [5]:
def optima_tt_beam_right(Y, k=100, ort_num=0, ret_all=False):
    ort_num = ort_num if ort_num >= 0 else len(Y)-1
    Z = teneva.orthogonalize(Y, ort_num)
    
    G = Z[0]
    r1, n, r2 = G.shape
    Q = G.reshape(n, r2)
    I = _range(n)

    for G in Z[1:]:
        r1, n, r2 = G.shape
        Q = np.einsum('ir,rjq->ijq', Q, G).reshape(-1, r2)
        
        I1 = np.kron(I, _ones(n))
        I2 = np.kron(_ones(I.shape[0]), _range(n))
        I = np.hstack((I1, I2))
        
        norms = np.sum(Q**2, axis=1)
        ind = np.argsort(norms)[:-(k+1):-1]
        I = I[ind]
        Q = Q[ind]

    return I if ret_all else I[0]

# Test 1

In [6]:
t = tpc()

FUNCS = []

for func in teneva.func_demo_all(d=5):
    if func.name == 'Brown': continue # We do not like this function!
    func.set_grid(n=16, kind='uni')
    
    I_full_real = teneva.grid_flat(func.n)
    Y_full_real = func.get_f_ind(I_full_real).reshape(func.n, order='F')
    
    Y = teneva.svd(Y_full_real, e=1.E-10)
    # Y = teneva.truncate(Y, e=1.E-10)
    
    Y_full = teneva.full(Y)
    i_min = np.unravel_index(np.argmin(Y_full), Y_full.shape)
    i_max = np.unravel_index(np.argmax(Y_full), Y_full.shape)
    
    FUNCS.append({
        'Y': Y,
        'name': func.name,
        'r': teneva.erank(Y),
        'i_max': i_max,
        'i_min': i_min,
        'y_max': Y_full[i_max],
        'y_min': Y_full[i_min],
    })

print(f'DONE   | FUNCS | Total time : {tpc()-t:-7.4f} sec.')

DONE   | FUNCS | Total time :  6.4659 sec.


In [7]:
e = 1.E-8

for ort_num_r in [0, 4]:
    for ort_num_l in [4, 0]:

        print(f'\nTest 1 | Ort -> : {ort_num_r} | Ort <- : {ort_num_l} |\n')

        for item in FUNCS:
            if item['name'] == 'Brown': continue # We do not like this function!

            Y = item['Y']
            # Y = teneva.truncate(Y, e)

            i_ref = item['i_min'] if abs(item['y_min']) < abs(item['y_max']) else item['i_max']
            y_ref = item['y_min'] if abs(item['y_min']) < abs(item['y_max']) else item['y_max']

            # Left to right :

            i_max = optima_tt_beam_right(Y, ort_num=ort_num_r)
            y_max = teneva.get(Y, i_max)

            D = teneva.tensor_const(teneva.shape(Y), y_max)
            Z = teneva.sub(Y, D)
            # Z = teneva.truncate(Z, 1.E-10)
            
            i_min = optima_tt_beam_right(Z, ort_num=ort_num_r)
            y_min = teneva.get(Z, i_min) + y_max

            di_r = np.sum(np.abs(i_ref - i_min))
            dy_r = abs(y_ref - y_min) if di_r > 0 else 0.

            # Right to left :

            i_max = optima_tt_beam_left(Y, ort_num=ort_num_l)
            y_max = teneva.get(Y, i_max)

            D = teneva.tensor_const(teneva.shape(Y), y_max)
            Z = teneva.sub(Y, D)
            # Z = teneva.truncate(Z, 1.E-10)
            
            i_min = optima_tt_beam_left(Z, ort_num=ort_num_l)
            y_min = teneva.get(Z, i_min) + y_max

            di_l = np.sum(np.abs(i_ref - i_min))
            dy_l = abs(y_ref - y_min) if di_l > 0 else 0.

            # Log result:
            
            text = item['name'] + ' ' * max(0, 15 - len(item['name'])) + f' | '
            text += f'di/dy -> : {di_r:-4d} / {dy_r:-7.1e} | '
            text += f'di/dy <- : {di_l:-4d} / {dy_l:-7.1e} | '
            text += f'dy best : {min(dy_l, dy_r):-7.1e}'
            if min(dy_l, dy_r) > 1.E-4:
                text += ' !!!'
            print(text)


Test 1 | Ort -> : 0 | Ort <- : 4 |

Ackley          | di/dy -> :    2 / 7.1e-15 | di/dy <- :    1 / 7.1e-15 | dy best : 7.1e-15
Alpine          | di/dy -> :    0 / 0.0e+00 | di/dy <- :    0 / 0.0e+00 | dy best : 0.0e+00
Dixon           | di/dy -> :    0 / 0.0e+00 | di/dy <- :    0 / 0.0e+00 | dy best : 0.0e+00
Exponential     | di/dy -> :   30 / 1.8e-16 | di/dy <- :   15 / 1.8e-16 | dy best : 1.8e-16
Grienwank       | di/dy -> :    1 / 2.6e-13 | di/dy <- :    0 / 0.0e+00 | dy best : 0.0e+00
Michalewicz     | di/dy -> :   31 / 2.9e-16 | di/dy <- :   30 / 2.9e-16 | dy best : 2.9e-16
Qing            | di/dy -> :    0 / 0.0e+00 | di/dy <- :    0 / 0.0e+00 | dy best : 0.0e+00
Rastrigin       | di/dy -> :    3 / 8.4e-14 | di/dy <- :    3 / 8.4e-14 | dy best : 8.4e-14
Rosenbrock      | di/dy -> :   12 / 4.4e+00 | di/dy <- :    0 / 0.0e+00 | dy best : 0.0e+00
Schaffer        | di/dy -> :    3 / 9.4e-16 | di/dy <- :    2 / 5.0e-16 | dy best : 5.0e-16
Schwefel        | di/dy -> :    0 / 0.0e+00

# Test 2

In [8]:
d = 100
n = 10
r = 2
e = 1.E-16
y_max_scale = 0.1
i_max_ref = [4] * d
y_max_ref = 1. + y_max_scale

In [9]:
for ort_num_r in [0, d-1]:
    for ort_num_l in [d-1, 0]:

        print(f'\nTest 2 | Ort -> : {ort_num_r} | Ort <- : {ort_num_l} |\n')

        for it in range(5):
            Y = teneva.tensor_rand([n]*d, r)
            # Y = teneva.truncate(Y, e)

            # Left to right :

            i_max_pre = optima_tt_beam_right(Y, ort_num=ort_num_r)
            y_max_pre = teneva.get(Y, i_max_pre)

            Z = teneva.mul(Y, 1./y_max_pre)
            Z = teneva.truncate(Z, e)

            y = teneva.get(Z, i_max_ref)
            D = teneva.tensor_delta([n]*d, i_max_ref, y_max_ref - y)
            Z = teneva.add(Z, D)
            # Z = teneva.truncate(Z, e)

            t = tpc()
            i_max = optima_tt_beam_right(Z, ort_num=ort_num_r)
            y_max = teneva.get(Z, i_max)
            t = tpc() - t

            di_r = np.sum(np.abs(i_max_ref - i_max))
            dy_r = abs(y_max_ref - y_max)

            # Right to left :

            i_max_pre = optima_tt_beam_left(Y, ort_num=ort_num_l)
            y_max_pre = teneva.get(Y, i_max_pre)

            Z = teneva.mul(Y, 1./y_max_pre)
            # Z = teneva.truncate(Z, e)

            y = teneva.get(Z, i_max_ref)
            D = teneva.tensor_delta([n]*d, i_max_ref, y_max_ref - y)
            Z = teneva.add(Z, D)
            # Z = teneva.truncate(Z, e)

            t = tpc()
            i_max = optima_tt_beam_left(Z, ort_num=ort_num_l)
            y_max = teneva.get(Z, i_max)
            t = tpc() - t

            di_l = np.sum(np.abs(i_max_ref - i_max))
            dy_l = abs(y_max_ref - y_max)

            # Log result:

            text = f'# {it:-3d} | '
            text += f'di/dy -> : {di_r:-4d} / {dy_r:-7.1e} | '
            text += f'di/dy <- : {di_l:-4d} / {dy_l:-7.1e} | '
            text += f'dy best : {min(dy_l, dy_r):-7.1e}'
            if min(dy_l, dy_r) > 1.E-4:
                text += ' !!!'
            print(text)


Test 2 | Ort -> : 0 | Ort <- : 99 |

#   0 | di/dy -> :  242 / 1.0e-01 | di/dy <- :  248 / 1.0e-01 | dy best : 1.0e-01 !!!
#   1 | di/dy -> :  255 / 1.0e-01 | di/dy <- :  257 / 1.0e-01 | dy best : 1.0e-01 !!!
#   2 | di/dy -> :  243 / 1.0e-01 | di/dy <- :  247 / 1.0e-01 | dy best : 1.0e-01 !!!
#   3 | di/dy -> :  222 / 1.0e-01 | di/dy <- :  224 / 1.0e-01 | dy best : 1.0e-01 !!!
#   4 | di/dy -> :  260 / 1.0e-01 | di/dy <- :  255 / 1.0e-01 | dy best : 1.0e-01 !!!

Test 2 | Ort -> : 0 | Ort <- : 0 |

#   0 | di/dy -> :  241 / 1.0e-01 | di/dy <- :    8 / 1.1e+00 | dy best : 1.0e-01 !!!
#   1 | di/dy -> :  263 / 1.0e-01 | di/dy <- :    8 / 1.1e+00 | dy best : 1.0e-01 !!!
#   2 | di/dy -> :  245 / 1.0e-01 | di/dy <- :   12 / 1.1e+00 | dy best : 1.0e-01 !!!
#   3 | di/dy -> :  248 / 1.0e-01 | di/dy <- :    7 / 1.1e+00 | dy best : 1.0e-01 !!!
#   4 | di/dy -> :  252 / 1.0e-01 | di/dy <- :    9 / 1.1e+00 | dy best : 1.0e-01 !!!

Test 2 | Ort -> : 99 | Ort <- : 99 |

#   0 | di/dy -> :    0 / 

# Test 3

In [37]:
func = teneva.FuncDemoGrienwank(d=5)
func.set_grid(n=1024)
func.build_vld_ind(m=1.E+3)
func.build_tst_ind(m=1.E+4)
func.rand(r=1)
func.cross(m=1.E+8, e_vld=1.E-10, log=True)
func.check()
func.info()

# pre | time:      0.847 | evals: 0.00e+00 (+ 0.00e+00) | rank:   1.0 | err: 1.0e+00 | 
#   1 | time:      2.431 | evals: 3.07e+04 (+ 2.08e+03) | rank:   3.0 | err: 2.1e-01 | eps: 2.7e+02 | 
#   2 | time:      6.055 | evals: 1.37e+05 (+ 1.96e+04) | rank:   5.8 | err: 3.9e-15 | eps: 1.8e-01 | stop: e_vld | 

Grienwank       [CRO          ] > error: 5.5e-15 / 4.2e-15 / 4.3e-15 | rank:  3.0 | time:   6.061


In [59]:
Y = func.Y
i_ref = teneva.poi_to_ind(func.x_min, func.a, func.b, func.n)
y_ref = teneva.get(Y, i_ref)
print(i_ref)
print(y_ref)

[512 512 512 512 512]
0.660843894011238


In [60]:
def run(Y, func, ort_num, i_ref, y_ref, is_max=False):
    i_max = func(Y, ort_num=ort_num)
    y_max = teneva.get(Y, i_max)

    if is_max:
        i_min = i_max
        y_min = y_max
    else:
        D = teneva.tensor_const(teneva.shape(Y), y_max)
        Z = teneva.sub(Y, D)
        # Z = teneva.truncate(Z, 1.E-10)

        i_min = func(Z, ort_num=ort_num)
        y_min = teneva.get(Z, i_min) + y_max
    print(y_min, y_max)
    di = np.sum(np.abs(i_ref - i_min))
    dy = abs(y_ref - y_min) if di > 0 else 0.
    return di, dy

In [61]:
# Left to right :
di_r, dy_r = run(Y, optima_tt_beam_right, -1, i_ref, y_ref)

# Right to left :
di_l, dy_l = run(Y, optima_tt_beam_left, 0, i_ref, y_ref)

# Log result:
text = func.name + ' ' * max(0, 15 - len(func.name)) + f' | '
text += f'di/dy -> : {di_r:-4d} / {dy_r:-7.1e} | '
text += f'di/dy <- : {di_l:-4d} / {dy_l:-7.1e} | '
text += f'dy best : {min(dy_l, dy_r):-7.1e}'
if min(dy_l, dy_r) > 1.E-4:
    text += ' !!!'
print(text)

338.10298570775916 449.3405234170433
450.9948725271845 94.38462198614768
Grienwank       | di/dy -> : 1828 / 3.4e+02 | di/dy <- : 2556 / 4.5e+02 | dy best : 3.4e+02 !!!


In [62]:
def _ones(n):
    return np.ones((n, 1), dtype=int)

In [63]:
def _range(n):
    return np.arange(n).reshape(-1, 1)

In [64]:
def _reshape(A, shape):
    # return A.reshape(shape, order='F')
    return A.reshape(shape)

In [67]:
def optima_tt_beam(Y, k=200, core_only=None, log=False):
    d = len(Y)
    get = teneva.getter(Y)

    i_opt = None
    y_opt = None
    
    for m in range(d):
        if core_only is not None and m != core_only:
            continue

        Z = teneva.copy(Y)
        teneva.orthogonalize(Z, m)

        if m > 0:
            G = Z[m].copy()
            r1, n, r2 = G.shape
            Ql = _reshape(G, (r1, n*r2))
            Il = np.kron(_ones(r2), _range(n))

            for m2 in range(m-1, -1, -1):
                G = Z[m2].copy()
                r1, n, r2 = G.shape
                Ql = np.einsum('qir,rj->qij', G, Ql)
                Ql = _reshape(Ql, (r1, -1))

                Il_ext = np.kron(_ones(n), Il)
                Il_new = np.kron(_range(n), _ones(Il.shape[0]))
                Il = np.hstack((Il_new, Il_ext))

                ind = np.argsort(np.sum(Ql**2, axis=0))[:-(k+1):-1]
                Ql = Ql[:, ind]
                Il = Il[ind, :]

        if m < d-1:
            G = Z[m].copy()
            r1, n, r2 = G.shape
            Qr = _reshape(G, (r1*n, r2))
            Ir = np.kron(_range(n), _ones(r1))
            
            for m2 in range(m+1, d):
                G = Z[m2].copy()
                r1, n, r2 = G.shape

                Qr = np.einsum('ir,rjq->ijq', Qr, G)
                Qr = _reshape(Qr, (-1, r2))

                Ir_ext = np.kron(Ir, _ones(n))
                Ir_new = np.kron(_ones(Ir.shape[0]), _range(n))
                Ir = np.hstack((Ir_ext, Ir_new))

                ind = np.argsort(np.sum(Qr**2, axis=1))[:-(k+1):-1]
                Qr = Qr[ind, :]
                Ir = Ir[ind, :]

        if m == 0:
            I = Ir.copy()
        elif m == d-1:
            I = Il.copy()
        else:
            I = []
            for il in Il:
                for ir in Ir:
                    I.append(np.hstack((il[:-1], ir)))
                    I.append(np.hstack((il, ir[1:])))
            I = np.array(I, dtype=int)

        y = [teneva.get(Y, i) for i in I]
        ind = np.argmax(np.abs(y))
        i_new = I[ind]
        y_new = y[ind]

        i_new = I[0]
        y_new = teneva.get(Y, i_new)

        if y_opt is None or abs(y_new) > abs(y_opt):
            i_opt = i_new.copy()
            y_opt = y_new
            
            if log:
                print(f'TT-core # {m:-3d} | y_opt: {y_opt:-16.9e}')

    return i_opt

In [68]:
def run2(Y, i_ref, y_ref, is_max=False):
    i_max = optima_tt_beam(Y)
    y_max = teneva.get(Y, i_max)

    if is_max:
        i_min = i_max
        y_min = y_max
    else:
        D = teneva.tensor_const(teneva.shape(Y), y_max)
        Z = teneva.sub(Y, D)
        # Z = teneva.truncate(Z, 1.E-10)

        i_min = optima_tt_beam(Z)
        y_min = teneva.get(Z, i_min) + y_max
    print(y_min, y_max)
    di = np.sum(np.abs(i_ref - i_min))
    dy = abs(y_ref - y_min) if di > 0 else 0.
    return di, dy

In [69]:
# Left to right :
di_r, dy_r = run2(Y, i_ref, y_ref)

# Right to left :
di_l, dy_l = di_r, dy_r

# Log result:
text = func.name + ' ' * max(0, 15 - len(func.name)) + f' | '
text += f'di/dy -> : {di_r:-4d} / {dy_r:-7.1e} | '
text += f'di/dy <- : {di_l:-4d} / {dy_l:-7.1e} | '
text += f'dy best : {min(dy_l, dy_r):-7.1e}'
if min(dy_l, dy_r) > 1.E-4:
    text += ' !!!'
print(text)

1.1642163271931736 450.9959786735153
Grienwank       | di/dy -> :   63 / 5.0e-01 | di/dy <- :   63 / 5.0e-01 | dy best : 5.0e-01 !!!


In [70]:
i_opt = optima_tt_beam(Y, k=100, log=True)

TT-core #   0 | y_opt:  4.509959787e+02


In [36]:
y_ref = teneva.get(Y, i_ref)
print(y_ref)

0.7448211122939488


---